# Glicko-2 Reward Function for Nectar Dataset

This notebook implements a reward function based on Glicko-2 ratings for the Berkeley NEST Nectar dataset. The reward function is defined as:

```
Reward = Glicko-2 Rating - Rating Volatility
```

This rewards items with high ratings while penalizing those with high volatility (inconsistency).

In [1]:
# -*- coding: utf-8 -*-
import time
import math
from collections import defaultdict
from datetime import datetime
import pandas as pd
from datasets import load_dataset, IterableDataset, concatenate_datasets # Make sure IterableDataset is imported if using streaming
from glicko2 import Player
import re

In [3]:
d = load_dataset("Columbia-NLP/DPO-tldr-summarisation-preferences")
train = d['train']
val = d['validation']
test = d['test']
dataset = train.add_column("sub_reddit", [x['subreddit'] for x in train['other_info']])
dataset_test = val.add_column("sub_reddit", [x['subreddit'] for x in val['other_info']])
dataset_eval = test.add_column("sub_reddit", [x['subreddit'] for x in test['other_info']])

In [4]:
start_time = time.time()

In [5]:
p2i = {}
i2p = {}
a2i = {}
i2a = {}
p = 0 
a = 0 
for row in dataset:
    if row['prompt'] not in p2i:
        p2i[row['prompt']] = p
        i2p[p] = row['prompt']
        p+=1
    if row['chosen'][1]['content'] not in a2i:
        a2i[row['chosen'][1]['content']] = a
        i2a[a] = row['chosen'][1]['content']
        a += 1
    if row['rejected'][1]['content'] not in a2i:
        a2i[row['rejected'][1]['content']] = a
        i2a[a] = row['rejected'][1]['content']
        a += 1
for row in dataset_test:
    if row['prompt'] not in p2i:
        p2i[row['prompt']] = p
        i2p[p] = row['prompt']
        p+=1
    if row['chosen'][1]['content'] not in a2i:
        a2i[row['chosen'][1]['content']] = a
        i2a[a] = row['chosen'][1]['content']
        a += 1
    if row['rejected'][1]['content'] not in a2i:
        a2i[row['rejected'][1]['content']] = a
        i2a[a] = row['rejected'][1]['content']
        a += 1

In [6]:
# --- 3. Initialize Player Storage ---
print("Initializing player storage...")
# Use the Player class defined above
players = {} # Dictionary to store author_id -> Player object instance
all_known_players = set() # Keep track of all players ever encountered

def get_player(author_id):
    """Gets or creates a Player object for a valid author ID."""
    # Check specifically against None and handle empty strings if necessary
    if author_id is not None and author_id.strip() != '' and author_id.lower() != '[deleted]':
        if author_id not in players:
            players[author_id] = Player() # Use the provided Player class
            # print(f"Created player: {author_id}") # Optional: for debugging
        all_known_players.add(author_id) # Track all valid players encountered
        return players.get(author_id)
    return None # Return None for invalid authors

Initializing player storage...


In [7]:
# --- 4. Process Matches and Group by Period ---
print("Processing matches and grouping by period...")
matches_by_period = defaultdict(lambda: defaultdict(lambda: {'ratings': [], 'rds': [], 'outcomes': []}))
processed_count = 0
# max_rows_to_process = 100000 # Optional: Limit rows for testing even without streaming
skipped_same_author = 0
skipped_deleted = 0
skipped_fetch_error = 0
max_rows_to_process = 5000
# Now iterate directly over the loaded dataset (which acts like a list/dict)
try:
    for i, row in enumerate(dataset):
        processed_count += 1
        # if processed_count > max_rows_to_process: # Apply optional limit if defined
        #      print(f"Reached processing limit of {max_rows_to_process} rows.")
        #      break
        
        # Extract data
        p_ind = p2i[row.get('prompt')]
        # Player j won
        author_j = str((p_ind, a2i[row.get('chosen')[1]['content']]))
        author_k = str((p_ind, a2i[row.get('rejected')[1]['content']]))

        if author_j is None or author_k is None:
            skipped_fetch_error += 1
            continue
        if author_j == author_k:
            skipped_same_author += 1
            continue

        player_j = get_player(author_j)
        player_k = get_player(author_k)

        if player_j is None or player_k is None:
            skipped_deleted += 1
            continue
        period = '0'
        rating_j_current = player_j.rating
        rd_j_current = player_j.rd
        rating_k_current = player_k.rating
        rd_k_current = player_k.rd

        outcome_for_j = 1.0
        outcome_for_k = 0.0

        matches_by_period[period][author_j]['ratings'].append(rating_k_current)
        matches_by_period[period][author_j]['rds'].append(rd_k_current)
        matches_by_period[period][author_j]['outcomes'].append(outcome_for_j)

        matches_by_period[period][author_k]['ratings'].append(rating_j_current)
        matches_by_period[period][author_k]['rds'].append(rd_j_current)
        matches_by_period[period][author_k]['outcomes'].append(outcome_for_k)

        if processed_count % 20000 == 0: # Adjust print frequency as needed
            print(f"Gathered data for {processed_count}/{len(dataset)} rows...")

except Exception as e:
     print(f"An error occurred during data iteration at row {processed_count}: {e}")


print(f"\n--- Data Gathering Summary ---")
# ... [Summary print statements remain the same] ...
print(f"Total rows processed: {processed_count}")
print(f"Matches skipped (same author): {skipped_same_author}")
print(f"Matches skipped (deleted/invalid author): {skipped_deleted}")
print(f"Matches skipped (data fetch error): {skipped_fetch_error}")
print(f"Total unique valid authors encountered: {len(all_known_players)}")
print(f"Number of rating periods found: {len(matches_by_period)}")


Processing matches and grouping by period...
Gathered data for 20000/92858 rows...
Gathered data for 40000/92858 rows...
Gathered data for 60000/92858 rows...
Gathered data for 80000/92858 rows...

--- Data Gathering Summary ---
Total rows processed: 92858
Matches skipped (same author): 12
Matches skipped (deleted/invalid author): 0
Matches skipped (data fetch error): 0
Total unique valid authors encountered: 60672
Number of rating periods found: 1


In [8]:
# --- 4. Process Matches and Group by Period for test set---
# Now iterate directly over the loaded dataset (which acts like a list/dict)
try:
    for i, row in enumerate(dataset_test):
        processed_count += 1
        # if processed_count > max_rows_to_process: # Apply optional limit if defined
        #      print(f"Reached processing limit of {max_rows_to_process} rows.")
        #      break

        # Extract data
        p_ind = p2i[row.get('prompt')]
        author_j = str((p_ind, a2i[row.get('chosen')[1]['content']]))
        author_k = str((p_ind, a2i[row.get('rejected')[1]['content']]))
        label = row.get('labels')

        if author_j is None or author_k is None:
            skipped_fetch_error += 1
            continue
        if author_j == author_k:
            skipped_same_author += 1
            continue

        player_j = get_player(author_j)
        player_k = get_player(author_k)

        if player_j is None or player_k is None:
            skipped_deleted += 1
            continue
        period = '0'
        rating_j_current = player_j.rating
        rd_j_current = player_j.rd
        rating_k_current = player_k.rating
        rd_k_current = player_k.rd

        # Player j won
        outcome_for_j = 1.0
        outcome_for_k = 0.0


        matches_by_period[period][author_j]['ratings'].append(rating_k_current)
        matches_by_period[period][author_j]['rds'].append(rd_k_current)
        matches_by_period[period][author_j]['outcomes'].append(outcome_for_j)

        matches_by_period[period][author_k]['ratings'].append(rating_j_current)
        matches_by_period[period][author_k]['rds'].append(rd_j_current)
        matches_by_period[period][author_k]['outcomes'].append(outcome_for_k)

        if processed_count % 20000 == 0: # Adjust print frequency as needed
            print(f"Gathered data for {processed_count}/{len(dataset)} rows...")

except Exception as e:
     print(f"An error occurred during data iteration at row {processed_count}: {e}")


print(f"\n--- Data Gathering Summary ---")
# ... [Summary print statements remain the same] ...
print(f"Total rows processed: {processed_count}")
print(f"Matches skipped (same author): {skipped_same_author}")
print(f"Matches skipped (deleted/invalid author): {skipped_deleted}")
print(f"Matches skipped (data fetch error): {skipped_fetch_error}")
print(f"Total unique valid authors encountered: {len(all_known_players)}")
print(f"Number of rating periods found: {len(matches_by_period)}")


Gathered data for 100000/92858 rows...
Gathered data for 120000/92858 rows...

--- Data Gathering Summary ---
Total rows processed: 125941
Matches skipped (same author): 12
Matches skipped (deleted/invalid author): 0
Matches skipped (data fetch error): 0
Total unique valid authors encountered: 77603
Number of rating periods found: 1


In [9]:
# --- 5. Run Glicko-2 Updates Period by Period ---
print("\nUpdating ratings period by period...")
# ... [Rating update loop remains the same as previous response] ...
sorted_periods = sorted(matches_by_period.keys())

for period in sorted_periods:
    print(f"Processing period: {period}")
    players_in_period = set(matches_by_period[period].keys())

    # Identify players who *didn't* compete in this period but existed before
    inactive_players_in_period = all_known_players - players_in_period

    # Apply Step 6: Update RD for inactive players
    inactive_update_count = 0
    for author_id in inactive_players_in_period:
        player = players.get(author_id)
        if player: # Should always exist if in all_known_players
             try:
                 player.did_not_compete()
                 inactive_update_count += 1
             except Exception as e:
                 print(f"Error calling did_not_compete for {author_id} in period {period}: {e}")

    # Apply Steps 3-5: Update rating, RD, vol for active players
    updates_count = 0
    for author_id in players_in_period:
        player = players.get(author_id)
        if not player: continue # Should not happen if key exists, but safe check

        period_data = matches_by_period[period][author_id]
        opponent_ratings = period_data['ratings']
        opponent_rds = period_data['rds']
        outcomes = period_data['outcomes']

        # Ensure we have opponents before calling update
        if opponent_ratings:
            try:
                player.update_player(opponent_ratings, opponent_rds, outcomes)
                updates_count += 1
            except OverflowError:
                 print(f"OverflowError encountered updating player {author_id} in period {period}. Skipping update.")
                 # This can happen with extreme rating differences or RDs. Might indicate need for parameter tuning or data cleaning.
                 # Consider logging player state: player.rating, player.rd, player.vol and opponent data
            except FloatingPointError as fpe:
                print(f"FloatingPointError encountered updating player {author_id} in period {period}: {fpe}. Skipping update.")
            except Exception as e:
                 print(f"Error updating player {author_id} in period {period}: {e}")


    print(f"-> Completed updates for {updates_count} active players. Applied inactivity update for {inactive_update_count} players.")


print("Rating updates complete.")





Updating ratings period by period...
Processing period: 0
-> Completed updates for 77603 active players. Applied inactivity update for 0 players.
Rating updates complete.


In [10]:
# --- 6. Display Results ---
print("\n--- Final Glicko-2 Ratings for Authors ---")
# ... [Result display remains the same as previous response] ...
author_ratings = []
skipped_final_rating = 0
for author_id, player in players.items():
    try:
        # Add check for NaN/Inf before appending
        rating_val = player.rating
        rd_val = player.rd
        vol_val = player.vol
        if not (math.isfinite(rating_val) and math.isfinite(rd_val) and math.isfinite(vol_val)):
            print(f"Warning: Non-finite values for player {author_id}. Rating={rating_val}, RD={rd_val}, Vol={vol_val}. Skipping.")
            skipped_final_rating += 1
            continue

        author_ratings.append({
            "author": author_id,
            "rating": rating_val, # Uses the property getter
            "deviation (RD)": rd_val, # Uses the property getter
            "volatility": vol_val
        })
    except Exception as e:
        print(f"Error retrieving final rating for {author_id}: {e}")
        skipped_final_rating += 1


if skipped_final_rating > 0:
    print(f"\nNote: Skipped retrieving final ratings for {skipped_final_rating} authors due to errors or non-finite values.")

# Sort by rating descending for readability
results_df = pd.DataFrame(author_ratings)

if not results_df.empty:
    # Ensure columns are numeric before sorting
    results_df['rating'] = pd.to_numeric(results_df['rating'], errors='coerce')
    results_df['deviation (RD)'] = pd.to_numeric(results_df['deviation (RD)'], errors='coerce')
    results_df['volatility'] = pd.to_numeric(results_df['volatility'], errors='coerce')
    results_df.dropna(subset=['rating'], inplace=True) # Remove rows where rating became NaN

if not results_df.empty:
    results_df = results_df.sort_values(by="rating", ascending=False)

    # Display top N and bottom N for brevity, or the whole list
    pd.set_option('display.max_rows', 100) # Show more rows if needed
    print(f"Displaying top {min(20, len(results_df))} and bottom {min(10, len(results_df))} authors:")
    print(results_df.head(20))
    if len(results_df) > 30:
        print("...")
        print(results_df.tail(10))
    elif len(results_df) > 20:
         print("...") # Just indicate truncation if list is moderately long
else:
    print("No valid final ratings could be generated or displayed.")




--- Final Glicko-2 Ratings for Authors ---
Displaying top 20 and bottom 10 authors:
               author       rating  deviation (RD)  volatility
35363   (8732, 35336)  1994.563819       76.398776    0.063120
30890   (7644, 30870)  1994.563819       76.398776    0.063120
35629   (8794, 35602)  1994.016832       77.239280    0.062963
47871  (11721, 47822)  1993.445098       78.108149    0.062811
48371  (11844, 48322)  1993.445098       78.108149    0.062811
49334  (12076, 49285)  1992.220354       79.937647    0.062522
26105   (6471, 26094)  1990.148941       82.940136    0.062126
13012   (3235, 13011)  1989.386177       84.018741    0.062003
17569   (4359, 17566)  1989.386177       84.018741    0.062003
11883   (2957, 11883)  1987.734235       86.308532    0.061770
17392   (4316, 17389)  1987.734235       86.308532    0.061770
16160   (4013, 16157)  1985.889097       88.796359    0.061555
15322   (3805, 15320)  1985.889097       88.796359    0.061555
19942   (4950, 19937)  1985.88909

In [11]:
# Convert to DataFrame
results_df = pd.DataFrame(author_ratings)

# --- Normalization Step ---
if not results_df.empty:
    print(f"\nNormalizing {len(results_df)} valid results...")
    # Min-Max Normalization (scaling to [0, 1])

    # Rating
    r_min = results_df['rating'].min()
    r_max = results_df['rating'].max()
    if (r_max - r_min) != 0:
        results_df['rating_norm'] = (results_df['rating'] - r_min) / (r_max - r_min)
    else:
        results_df['rating_norm'] = 0.5 # Assign midpoint if all values are the same

    # Deviation (RD)
    rd_min = results_df['deviation (RD)'].min()
    rd_max = results_df['deviation (RD)'].max()
    if (rd_max - rd_min) != 0:
        results_df['rd_norm'] = (results_df['deviation (RD)'] - rd_min) / (rd_max - rd_min)
    else:
        results_df['rd_norm'] = 0.5

    # Volatility
    v_min = results_df['volatility'].min()
    v_max = results_df['volatility'].max()
    if (v_max - v_min) != 0:
        results_df['volatility_norm'] = (results_df['volatility'] - v_min) / (v_max - v_min)
    else:
        results_df['volatility_norm'] = 0.5

    # Sort by original rating
    results_df = results_df.sort_values(by="rating", ascending=False)

    # Display selected columns including normalized ones
    pd.set_option('display.max_rows', 100)
    pd.set_option('display.width', 120) # Adjust width for better display
    print(f"\nDisplaying top {min(20, len(results_df))} authors (incl. normalized scores):")
    # Select columns to display
    display_cols = ['author', 'rating', 'rating_norm', 'deviation (RD)', 'rd_norm', 'volatility', 'volatility_norm']
    print(results_df[display_cols].head(20).round(4)) # Round for display

    if len(results_df) > 20:
        print("...")
        print(f"\nDisplaying bottom {min(10, len(results_df))} authors (incl. normalized scores):")
        print(results_df[display_cols].tail(min(10, max(0, len(results_df)))).round(4))

else:
    print("No valid final ratings could be generated or displayed.")
df = results_df
auth = df['author'].apply(lambda x: re.sub('[^A-Za-z0-9,]', '', str(x))).str.split(',')
prompts = auth.apply(lambda x: i2p[int(x[0])])
answers = auth.apply(lambda x: i2a[int(x[1])])
df['question'] = prompts
df['answer'] = answers

# Optionally, save to CSV (including normalized columns)
try:
    if not df.empty:
        df.to_csv("author_ratings_normalized.csv", index=False, float_format='%.6f')
        print("\nResults saved to author_ratings_normalized.csv")
except Exception as e:
    print(f"\nError saving results to CSV: {e}")
end_time = time.time()
print(f"\nCurrent time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Total execution time: {end_time - start_time:.2f} seconds")


Normalizing 77603 valid results...

Displaying top 20 authors (incl. normalized scores):
               author     rating  rating_norm  deviation (RD)  rd_norm  volatility  volatility_norm
35363   (8732, 35336)  1994.5638       1.0000         76.3988   0.0649      0.0631           0.3339
30890   (7644, 30870)  1994.5638       1.0000         76.3988   0.0649      0.0631           0.3339
35629   (8794, 35602)  1994.0168       0.9995         77.2393   0.0686      0.0630           0.3176
47871  (11721, 47822)  1993.4451       0.9989         78.1081   0.0724      0.0628           0.3018
48371  (11844, 48322)  1993.4451       0.9989         78.1081   0.0724      0.0628           0.3018
49334  (12076, 49285)  1992.2204       0.9977         79.9376   0.0804      0.0625           0.2718
26105   (6471, 26094)  1990.1489       0.9956         82.9401   0.0935      0.0621           0.2306
13012   (3235, 13011)  1989.3862       0.9948         84.0187   0.0983      0.0620           0.2178
17569   (4

In [12]:
# Save dictionaries that map text to indices
import pickle
def save_dictionary(dictionary, filename):
    """
    Save a dictionary to a file using pickle.
    
    Args:
        dictionary (dict): The dictionary to save
        filename (str): The name of the file to save to
    """
    with open(filename, 'wb') as file:
        pickle.dump(dictionary, file)
    print(f"Dictionary saved to {filename}")

In [13]:
save_dictionary(p2i, 'p2i')
save_dictionary(i2a, 'i2a')
save_dictionary(i2p, 'i2p')
save_dictionary(a2i, 'a2i')

Dictionary saved to p2i
Dictionary saved to i2a
Dictionary saved to i2p
Dictionary saved to a2i


In [14]:
results_df.describe()

,rating,deviation (RD),volatility,rating_norm,rd_norm,volatility_norm
count,77603.000000,77603.000000,77603.000000,77603.000000,77603.000000,77603.000000
mean,1496.322613,236.932846,0.060015,0.500665,0.766646,0.011047
std,215.695561,29.360423,0.000154,0.216169,0.128336,0.016053
min,996.753739,61.541146,0.059909,0.000000,0.000000,0.000000
25%,1252.681917,227.735397,0.059997,0.256490,0.726444,0.009154
50%,1500.000000,253.404596,0.060003,0.504351,0.838645,0.009782
75%,1747.318083,253.404608,0.060003,0.752212,0.838645,0.009782
max,1994.563819,290.318965,0.069525,1.000000,1.000000,1.000000


In [15]:
# --- 1. Provided Glicko-2 Player Class ---
class Player:
    # Class attribute
    # The system constant, which constrains
    # the change in volatility over time.
    _tau = 0.5

    def getRating(self):
        return (self.__rating * 173.7178) + 1500

    def setRating(self, rating):
        self.__rating = (rating - 1500) / 173.7178

    rating = property(getRating, setRating)

    def getRd(self):
        return self.__rd * 173.7178

    def setRd(self, rd):
        self.__rd = rd / 173.7178

    rd = property(getRd, setRd)

    def __init__(self, rating = 1500, rd = 350, vol = 0.06):
        # For testing purposes, preload the values
        # assigned to an unrated player.
        self.setRating(rating)
        self.setRd(rd)
        self.vol = vol

    def _preRatingRD(self):
        """ Calculates and updates the player's rating deviation for the
        beginning of a rating period.

        preRatingRD() -> None

        """
        self.__rd = math.sqrt(math.pow(self.__rd, 2) + math.pow(self.vol, 2))

    def update_player(self, rating_list, RD_list, outcome_list):
        """ Calculates the new rating and rating deviation of the player.

        update_player(list[int], list[int], list[bool]) -> None
        Docstring notes list[bool] for outcome, but expects numeric 0, 0.5, 1
        """
        # Convert the opponent rating and rating deviation values for internal use.
        rating_list = [(x - 1500) / 173.7178 for x in rating_list]
        RD_list = [x / 173.7178 for x in RD_list]

        v = self._v(rating_list, RD_list)
        self.vol = self._newVol(rating_list, RD_list, outcome_list, v)
        self._preRatingRD() # Applies Step 2 (modified RD) internally *after* vol update

        self.__rd = 1 / math.sqrt((1 / math.pow(self.__rd, 2)) + (1 / v))

        tempSum = 0
        for i in range(len(rating_list)):
            tempSum += self._g(RD_list[i]) * \
                       (outcome_list[i] - self._E(rating_list[i], RD_list[i]))
        self.__rating += math.pow(self.__rd, 2) * tempSum


    def _newVol(self, rating_list, RD_list, outcome_list, v):
        """ Calculating the new volatility as per the Glicko2 system.

        _newVol(list, list, list) -> float

        """
         # Convergence tolerance for the iteration
        CONVERGENCE_TOLERANCE = 0.000001
        i = 0
        delta = self._delta(rating_list, RD_list, outcome_list, v)
        a = math.log(math.pow(self.vol, 2))
        tau = self._tau
        # Use rating deviation (phi) from the start of the period for calculation
        pre_rating_rd_sq = math.pow(self.__rd, 2) + math.pow(self.vol, 2) # RD^2 + vol^2 before update

        # Simplified conditions for the Illinois algorithm based on Glickman's paper
        A = a
        if (delta**2 > pre_rating_rd_sq + v):
             B = math.log(delta**2 - pre_rating_rd_sq - v)
        else:
             k = 1
             while self._f(a - k * tau, delta, v, a, tau) < 0:
                  k = k + 1
             B = a - k * tau

        fA = self._f(A, delta, v, a, tau)
        fB = self._f(B, delta, v, a, tau)

        while math.fabs(B - A) > CONVERGENCE_TOLERANCE:
            C = A + (A - B) * fA / (fB - fA)
            fC = self._f(C, delta, v, a, tau)
            if fC * fB < 0:
                 A = B
                 fA = fB
            else:
                 fA = fA / 2.0
            B = C
            fB = fC

        return math.exp(A / 2.0)

    # Helper function for the volatility calculation's iterative method
    def _f(self, x, delta, v, a, tau):
        ex = math.exp(x)
        # Use rating deviation (phi) from the start of the period for calculation
        pre_rating_rd_sq = math.pow(self.__rd, 2) + math.pow(self.vol, 2) # RD^2 + vol^2 before update
        d_sq = pre_rating_rd_sq + v + ex # Denominator squared term estimate

        # Handle potential division by zero or log of non-positive if d_sq is problematic
        if d_sq <= 0: return -1 # Or some other indicator of error/boundary condition

        term1 = ex * (delta**2 - pre_rating_rd_sq - v - ex) / (2 * d_sq**2) if d_sq else 0
        term2 = (x - a) / tau**2
        return term1 - term2


    def _delta(self, rating_list, RD_list, outcome_list, v):
        """ The delta function of the Glicko2 system.

        _delta(list, list, list) -> float

        """
        tempSum = 0
        for i in range(len(rating_list)):
            tempSum += self._g(RD_list[i]) * (outcome_list[i] - self._E(rating_list[i], RD_list[i]))
        return v * tempSum

    def _v(self, rating_list, RD_list):
        """ The v function of the Glicko2 system.

        _v(list[int], list[int]) -> float

        """
        tempSum = 0
        for i in range(len(rating_list)):
            tempE = self._E(rating_list[i], RD_list[i])
            tempSum += math.pow(self._g(RD_list[i]), 2) * tempE * (1 - tempE)

        # Avoid division by zero if tempSum is zero (e.g., no opponents)
        if tempSum == 0:
            # Handle this case: perhaps return a very large number or raise error
            # Glickman's paper suggests V -> infinity, so 1/V -> 0.
            # Returning a very small inverse V, but update_player uses 1/V. Let's return a large V.
             return float('inf') # Or a very large number
        return 1 / tempSum

    def _E(self, p2rating, p2RD):
        """ The Glicko E function.

        _E(int) -> float

        """
        return 1 / (1 + math.exp(-1 * self._g(p2RD) * \
                                   (self.__rating - p2rating)))

    def _g(self, RD):
        """ The Glicko2 g(RD) function.

        _g() -> float

        """
        return 1 / math.sqrt(1 + 3 * math.pow(RD, 2) / math.pow(math.pi, 2))

    def did_not_compete(self):
        """ Applies Step 6 of the algorithm. Use this for
        players who did not compete in the rating period.

        did_not_compete() -> None

        """
        self._preRatingRD()